# Data preprocessing

In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from haversine import haversine
from pandas.tseries.holiday import USFederalHolidayCalendar
from datetime import datetime
%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
# NYC taxi. March, 2016
data = pd.read_csv(filepath_or_buffer='yellow_tripdata_2016-03.csv')
# Initial data dimensions
print(data.shape)
data.head()

(12210952, 19)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-03-01 00:00:00,2016-03-01 00:07:55,1,2.50,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,2016-03-01 00:00:00,2016-03-01 00:11:06,1,2.90,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2016-03-01 00:00:00,2016-03-01 00:31:06,2,19.98,-73.782021,40.644810,1,N,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,2,2016-03-01 00:00:00,2016-03-01 00:00:00,3,10.78,-73.863419,40.769814,1,N,-73.969650,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,2,2016-03-01 00:00:00,2016-03-01 00:00:00,5,30.43,-73.971741,40.792183,3,N,-74.177170,40.695053,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80


In [3]:
# Clear missing data
data = data.dropna(axis=0)
data.shape

(12210952, 19)

In [4]:
# Consider passengers count from 1 to 4
data = data.loc[(data.passenger_count <= 4) & (data.passenger_count >= 1)]
data.shape

(11160307, 19)

In [5]:
# Convert string to datetime
data['tpep_pickup_datetime'] = pd.to_datetime(data['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
data['tpep_dropoff_datetime'] = pd.to_datetime(data['tpep_dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')
# Clear invalid trips, i.e. dropoff earlier than pickup
data = data.loc[data.tpep_pickup_datetime < data.tpep_dropoff_datetime]
data.shape

(11147858, 19)

In [6]:
# Replace store_and_fwd_flag values Y or N to 1 and 0
data['store_and_fwd_flag'] = pd.Series(np.where(data.store_and_fwd_flag.values == 'Y', 1, 0), data.index)
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-03-01 00:00:00,2016-03-01 00:07:55,1,2.50,-73.976746,40.765152,1,0,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.0,0.3,12.35
1,1,2016-03-01 00:00:00,2016-03-01 00:11:06,1,2.90,-73.983482,40.767925,1,0,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.0,0.3,15.35
2,2,2016-03-01 00:00:00,2016-03-01 00:31:06,2,19.98,-73.782021,40.644810,1,0,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.0,0.3,63.80
7,1,2016-03-01 00:00:01,2016-03-01 00:16:04,1,6.20,-73.788773,40.647758,1,0,-73.829208,40.712345,3,20.5,0.5,0.5,0.00,0.0,0.3,21.80
8,1,2016-03-01 00:00:01,2016-03-01 00:05:00,1,0.70,-73.958221,40.764641,1,0,-73.967896,40.762901,1,5.5,0.5,0.5,2.00,0.0,0.3,8.80


In [7]:
# Since we have only 2 vendors, store it as boolean
data['VendorID'] = data['VendorID'].replace(2, 0)
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-03-01 00:00:00,2016-03-01 00:07:55,1,2.50,-73.976746,40.765152,1,0,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.0,0.3,12.35
1,1,2016-03-01 00:00:00,2016-03-01 00:11:06,1,2.90,-73.983482,40.767925,1,0,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.0,0.3,15.35
2,0,2016-03-01 00:00:00,2016-03-01 00:31:06,2,19.98,-73.782021,40.644810,1,0,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.0,0.3,63.80
7,1,2016-03-01 00:00:01,2016-03-01 00:16:04,1,6.20,-73.788773,40.647758,1,0,-73.829208,40.712345,3,20.5,0.5,0.5,0.00,0.0,0.3,21.80
8,1,2016-03-01 00:00:01,2016-03-01 00:05:00,1,0.70,-73.958221,40.764641,1,0,-73.967896,40.762901,1,5.5,0.5,0.5,2.00,0.0,0.3,8.80


In [8]:
# Checking data distribution
data.describe()

,VendorID,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
count,1.114786e+07,1.114786e+07,1.114786e+07,1.114786e+07,1.114786e+07,1.114786e+07,1.114786e+07,1.114786e+07,1.114786e+07,1.114786e+07,1.114786e+07,1.114786e+07,1.114786e+07,1.114786e+07,1.114786e+07,1.114786e+07,1.114786e+07
mean,5.126083e-01,1.309129e+00,6.431310e+00,-7.282865e+01,4.011992e+01,1.038893e+00,5.864804e-03,-7.297384e+01,4.020089e+01,1.340957e+00,1.279722e+01,3.466249e-01,4.976119e-01,1.793086e+00,3.139843e-01,2.997482e-01,1.604827e+01
std,4.998410e-01,6.572771e-01,6.443350e+03,9.133669e+00,5.031379e+00,3.715093e-01,7.635711e-02,8.542587e+00,4.705860e+00,4.915672e-01,1.403084e+02,4.567907e-01,4.961042e-02,2.558499e+00,1.772817e+00,1.181592e-02,1.405598e+02
min,0.000000e+00,1.000000e+00,0.000000e+00,-1.616987e+02,0.000000e+00,1.000000e+00,0.000000e+00,-1.616987e+02,0.000000e+00,1.000000e+00,-3.760000e+02,-5.850000e+01,-1.000000e+00,-2.000000e+01,-1.604000e+01,-3.000000e-01,-3.763000e+02
25%,0.000000e+00,1.000000e+00,1.000000e+00,-7.399176e+01,4.073623e+01,1.000000e+00,0.000000e+00,-7.399126e+01,4.073467e+01,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,8.750000e+00
50%,1.000000e+00,1.000000e+00,1.700000e+00,-7.398163e+01,4.075320e+01,1.000000e+00,0.000000e+00,-7.397964e+01,4.075376e+01,1.000000e+00,9.500000e+00,0.000000e+00,5.000000e-01,1.350000e+00,0.000000e+00,3.000000e-01,1.180000e+01
75%,1.000000e+00,1.000000e+00,3.170000e+00,-7.396680e+01,4.076759e+01,1.000000e+00,0.000000e+00,-7.396234e+01,4.076910e+01,2.000000e+00,1.450000e+01,5.000000e-01,5.000000e-01,2.360000e+00,0.000000e+00,3.000000e-01,1.776000e+01
max,1.000000e+00,4.000000e+00,1.907263e+07,0.000000e+00,6.685682e+01,9.900000e+01,1.000000e+00,0.000000e+00,5.079786e+01,4.000000e+00,4.294967e+05,8.000000e+01,7.930000e+01,8.500000e+02,1.410320e+03,3.000000e-01,4.295622e+05


In [9]:
# Difference between Min and Max of latitudes and longitudes are abnormally high
# but overall distribution is fine. Need to clean out outliers (Someone's traveling to Africa by taxi xD)
# We can suppose the distribution is normal and keep data in range of [mean-2*std,mean+2*std]

# Calculate means
# Pickup latitude
plat_mean = np.mean(data['pickup_latitude'])
# Pickup longitude
plong_mean = np.mean(data['pickup_longitude'])
# Dropoff latitude
dlat_mean = np.mean(data['dropoff_latitude'])
# Dropoff longitude
dlong_mean = np.mean(data['dropoff_longitude'])
plat_mean, plong_mean, dlat_mean, dlong_mean

(40.11992332164556, -72.82864711511783, 40.200891233735, -72.97383889519985)

In [10]:
# Calculate stds
# Pickup latitude
plat_std = np.std(data['pickup_latitude'])
# Pickup longitude
plong_std = np.std(data['pickup_longitude'])
# Dropoff latitude
dlat_std = np.std(data['dropoff_latitude'])
# Dropoff longitude
dlong_std = np.std(data['dropoff_longitude'])
plat_std, plong_std, dlat_std, dlong_std

(5.031378357006052, 9.133668100152104, 4.705859674341259, 8.54258612391909)

In [11]:
# Pickup latitude constraints
data = data.loc[(data.pickup_latitude >= (plat_mean-2*plat_std)) &
              (data.pickup_latitude <= (plat_mean+2*plat_std))]
# Pickup longitude constraints
data = data.loc[(data.pickup_longitude >= (plong_mean-2*plong_std)) &
                (data.pickup_longitude <= (plong_mean+2*plong_std))]
# Dropoff latitude constraints
data = data.loc[(data.dropoff_latitude >= (dlat_mean-2*dlat_std)) &
                (data.dropoff_latitude <= (dlat_mean+2*dlat_std))]
# Dropoff longitude constraints
data = data.loc[(data.dropoff_longitude >= (dlong_mean-2*dlong_std)) &
                (data.dropoff_longitude <= (dlong_mean+2*dlong_std))]
data.shape

(10969935, 19)

In [12]:
# Check data for second time
data.describe()

,VendorID,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
count,1.096994e+07,1.096994e+07,1.096994e+07,1.096994e+07,1.096994e+07,1.096994e+07,1.096994e+07,1.096994e+07,1.096994e+07,1.096994e+07,1.096994e+07,1.096994e+07,1.096994e+07,1.096994e+07,1.096994e+07,1.096994e+07,1.096994e+07
mean,5.093305e-01,1.309522e+00,6.491680e+00,-7.397384e+01,4.075088e+01,1.036387e+00,5.814164e-03,-7.397358e+01,4.075174e+01,1.340705e+00,1.277852e+01,3.468016e-01,4.977761e-01,1.790958e+00,3.141775e-01,2.997763e-01,1.602800e+01
std,4.999130e-01,6.575170e-01,6.495392e+03,4.399619e-02,3.129243e-02,3.586812e-01,7.602868e-02,4.144228e-02,3.497860e-02,4.912730e-01,1.414254e+02,4.558028e-01,4.342318e-02,2.543584e+00,1.738948e+00,1.119973e-02,1.416736e+02
min,0.000000e+00,1.000000e+00,0.000000e+00,-9.014469e+01,3.099193e+01,1.000000e+00,0.000000e+00,-8.515769e+01,3.225150e+01,1.000000e+00,-3.760000e+02,-4.500000e+00,-1.000000e+00,-1.306000e+01,-1.604000e+01,-3.000000e-01,-3.763000e+02
25%,0.000000e+00,1.000000e+00,1.000000e+00,-7.399192e+01,4.073744e+01,1.000000e+00,0.000000e+00,-7.399138e+01,4.073594e+01,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,8.750000e+00
50%,1.000000e+00,1.000000e+00,1.700000e+00,-7.398192e+01,4.075385e+01,1.000000e+00,0.000000e+00,-7.397996e+01,4.075432e+01,1.000000e+00,9.500000e+00,0.000000e+00,5.000000e-01,1.350000e+00,0.000000e+00,3.000000e-01,1.180000e+01
75%,1.000000e+00,1.000000e+00,3.180000e+00,-7.396787e+01,4.076790e+01,1.000000e+00,0.000000e+00,-7.396338e+01,4.076942e+01,2.000000e+00,1.450000e+01,5.000000e-01,5.000000e-01,2.360000e+00,0.000000e+00,3.000000e-01,1.775000e+01
max,1.000000e+00,4.000000e+00,1.907263e+07,-5.716621e+01,4.928364e+01,9.900000e+01,1.000000e+00,-5.715717e+01,4.836605e+01,4.000000e+00,4.294967e+05,8.000000e+01,7.930000e+01,8.500000e+02,1.410320e+03,3.000000e-01,4.295622e+05


In [13]:
# The range's still too big. Someone's still traveling across states
# Calculate means
# Pickup latitude
plat_mean = np.mean(data['pickup_latitude'])
# Pickup longitude
plong_mean = np.mean(data['pickup_longitude'])
# Dropoff latitude
dlat_mean = np.mean(data['dropoff_latitude'])
# Dropoff longitude
dlong_mean = np.mean(data['dropoff_longitude'])
plat_mean, plong_mean, dlat_mean, dlong_mean

# Calculate stds
# Pickup latitude
plat_std = np.std(data['pickup_latitude'])
# Pickup longitude
plong_std = np.std(data['pickup_longitude'])
# Dropoff latitude
dlat_std = np.std(data['dropoff_latitude'])
# Dropoff longitude
dlong_std = np.std(data['dropoff_longitude'])
plat_std, plong_std, dlat_std, dlong_std

# Pickup latitude constraints
data = data.loc[(data.pickup_latitude >= (plat_mean-2*plat_std)) &
              (data.pickup_latitude <= (plat_mean+2*plat_std))]
# Pickup longitude constraints
data = data.loc[(data.pickup_longitude >= (plong_mean-2*plong_std)) &
                (data.pickup_longitude <= (plong_mean+2*plong_std))]
# Dropoff latitude constraints
data = data.loc[(data.dropoff_latitude >= (dlat_mean-2*dlat_std)) &
                (data.dropoff_latitude <= (dlat_mean+2*dlat_std))]
# Dropoff longitude constraints
data = data.loc[(data.dropoff_longitude >= (dlong_mean-2*dlong_std)) &
                (data.dropoff_longitude <= (dlong_mean+2*dlong_std))]
print(data.shape)
# Check data for third time
data.describe()

(9733587, 19)


,VendorID,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
count,9.733587e+06,9.733587e+06,9.733587e+06,9.733587e+06,9.733587e+06,9.733587e+06,9.733587e+06,9.733587e+06,9.733587e+06,9.733587e+06,9.733587e+06,9.733587e+06,9.733587e+06,9.733587e+06,9.733587e+06,9.733587e+06,9.733587e+06
mean,5.109288e-01,1.307135e+00,5.466280e+00,-7.398131e+01,4.075291e+01,1.007576e+00,5.016239e-03,-7.397910e+01,4.075325e+01,1.342629e+00,1.013240e+01,3.350056e-01,4.989960e-01,1.414181e+00,2.686943e-02,2.997812e-01,1.270723e+01
std,4.998806e-01,6.562046e-01,6.706628e+03,1.699364e-02,2.143593e-02,2.804852e-01,7.064755e-02,1.929896e-02,2.404186e-02,4.902224e-01,4.302532e+01,3.815129e-01,3.683857e-02,1.584385e+00,6.554465e-01,1.112643e-02,4.318772e+01
min,0.000000e+00,1.000000e+00,0.000000e+00,-7.406146e+01,4.068830e+01,1.000000e+00,0.000000e+00,-7.405645e+01,4.068179e+01,1.000000e+00,-3.000000e+02,-4.500000e+00,-5.000000e-01,-1.306000e+01,-1.250000e+01,-3.000000e-01,-3.008000e+02
25%,0.000000e+00,1.000000e+00,9.600000e-01,-7.399255e+01,4.073877e+01,1.000000e+00,0.000000e+00,-7.399191e+01,4.073822e+01,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,8.300000e+00
50%,1.000000e+00,1.000000e+00,1.530000e+00,-7.398264e+01,4.075379e+01,1.000000e+00,0.000000e+00,-7.398123e+01,4.075462e+01,1.000000e+00,8.500000e+00,0.000000e+00,5.000000e-01,1.250000e+00,0.000000e+00,3.000000e-01,1.116000e+01
75%,1.000000e+00,1.000000e+00,2.550000e+00,-7.397105e+01,4.076673e+01,1.000000e+00,0.000000e+00,-7.396778e+01,4.076859e+01,2.000000e+00,1.250000e+01,5.000000e-01,5.000000e-01,2.160000e+00,0.000000e+00,3.000000e-01,1.535000e+01
max,1.000000e+00,4.000000e+00,1.907263e+07,-7.388586e+01,4.081347e+01,9.900000e+01,1.000000e+00,-7.389070e+01,4.082170e+01,4.000000e+00,1.330654e+05,6.553000e+01,7.930000e+01,4.402500e+02,8.125000e+02,3.000000e-01,1.331312e+05


In [14]:
data.to_csv('preprocessed_data.csv', index=False)